In [105]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from datetime import datetime


Webull scrapes from:
- yahoo finance
- bloomberg
- benziga
- reuters
- seeking alpha
- american city business journals
- zacks
- Dow jones
- Barron’s
- The Wall Street journal
- Tip ranks
- MT newswires
- NASDAQ
- Investor place
- MarkerWatch
- Motley Fool

# scraping using headlines (use this one)

In [106]:
def scrape_webull_headlines(ticker, date_tracker, start_phrase, end_phrase, scraped_headers, scraped_dates, file_name):

    # set up the webdriver
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)

    # navigate to the webpage
    driver.get("https://www.webull.com/newslist/nasdaq-" + ticker)

    continue_scrape = False
    # date_tracker = '03/27' #start date

    while True:
        
        try:
            more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
                                    (By.XPATH, "//div[@class='csr19'][text()='More']")))
            more_button.click()
            time.sleep(1)

            headers = driver.find_elements_by_class_name("csr14")
            dates = driver.find_elements_by_class_name("csr15")

            for i in range(len(headers)):
                header = headers[i]
                date = dates[i]

                # extract relevant items
                header = header.text
                date = date.text.split('·')[1].strip()
                date = date.split(' ')[0]

                if (len(date) == 10 and date[2] == '/' and date[5] == '/'):

                    date_tracker_obj = datetime.strptime(date_tracker, '%m/%d/%Y')
                    date_obj = datetime.strptime(str(date), '%m/%d/%Y')

                    if header == start_phrase and date_obj <= date_tracker_obj and header not in scraped_headers:
                        continue_scrape = True
                        scraped_headers.append(header)
                        scraped_dates.append(date)
                        date_tracker = str(date)
                        print("Scraping starts!...", header, date)

                    if header == end_phrase:
                        scraped_headers.append(header)
                        scraped_dates.append(date)
                        print("Scraping ends!...", header, date)
                        driver.quit()
                        break

                    if continue_scrape and date_obj <= date_tracker_obj and header not in scraped_headers:
                        scraped_headers.append(header)
                        scraped_dates.append(date)
                        date_tracker = str(date)
                        print("Scraping...", header, date)
        except:
            driver.quit()
            break
    
    # Combine the two lists into a DataFrame
    df = pd.DataFrame({'date': scraped_dates, 'news_headline': scraped_headers})

    # Save the DataFrame as a CSV file
    df.to_csv(file_name, index=False)

In [108]:
# run the function

# 399 minutes
# last 2022 post (dec 30): Tracking Philippe Laffont's Coatue Management Portfolio - Q3 2022 Update
# first 2022 post (jan 1): 3 Growth Stocks Poised for a Bull Run in 2022

# last 2021 post (dec 31):  Cathie Wood's Ark Continues To Book Profit In Tesla — These Are The Stocks It Bought On Thursday
# first 2021 post (jan 4): Slinger Bag Has Created A Product That Allows Tennis Players To Practice Anytime, Anywhere

# 408m: 6h
# last 2020 post (dec 31):  Tesla leads list of 2020's biggest winners on both S&P 500, Nasdaq 100
# first 2020 post (jan 3): Last year's Bay Area IPO performances show why it pays to be an insider


start_phrase = "California State University campuses to be closed fall semester" 
end_phrase =  "Last year's Bay Area IPO performances show why it pays to be an insider"

scraped_headers = []
scraped_dates = []
file_name = '2020_continue_full_webull_zoom_stock_news.csv'
ticker = 'zm'
date_tracker = '05/12/2020'

scrape_webull_headlines(ticker, date_tracker, start_phrase, end_phrase, scraped_headers, scraped_dates, file_name)

/var/folders/cf/28wm9krx5816hxprmj3lstxh0000gn/T/ipykernel_11467/4035272600.py:22: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  headers = driver.find_elements_by_class_name("csr14")
/var/folders/cf/28wm9krx5816hxprmj3lstxh0000gn/T/ipykernel_11467/4035272600.py:23: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  dates = driver.find_elements_by_class_name("csr15")


Scraping starts!... Tesla leads list of 2020's biggest winners on both S&P 500, Nasdaq 100 12/31/2020
Scraping... Will Zoom Or Cisco Stock Grow More By 2022? 12/31/2020
Scraping... Zoom Video, Cloudflare, and Fastly led tech gainers as work from home defined 2020 12/31/2020
Scraping... Zoom users push back against company's bid to axe privacy case 12/31/2020
Scraping... 2021 Predictions From Loup Ventures: Apple First Among FAANG, A Tesla Robotaxi, New Peloton Equipment 12/31/2020
Scraping... Zoom, Shopify and Nvidia Were Tech Winners in 2020. What Will Win in 2021? 12/31/2020
Scraping... The same stocks keep making the year's winners and losers list 12/30/2020
Scraping... Apple stock's stunning 2020 in three stats 12/29/2020
Scraping... The Zacks Analyst Blog Highlights: TSLA, JPM, FDX, ENPH and ZM 12/29/2020
Scraping... Margin Debt Reaches New High 12/28/2020
Scraping... Zoom (ZM) Explores Email, Messaging, Calendar Services 12/28/2020
Scraping... 3 Tech Stocks to Watch in 2021 12/28

# scraping using date (does not work)

In [81]:
def scrape_webull_headlines(ticker, start_date, end_date, scraped_headers, scraped_dates, file_name):

    # set up the webdriver
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)

    # navigate to the webpage
    driver.get("https://www.webull.com/newslist/nasdaq-" + ticker)

    continue_scrape = False
    date_tracker = start_date #start date

    while True:
        
        try:
            more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
                                    (By.XPATH, "//div[@class='csr19'][text()='More']")))
            more_button.click()
            time.sleep(1)

            headers = driver.find_elements_by_class_name("csr14")
            dates = driver.find_elements_by_class_name("csr15")

            for i in range(len(headers)):
                header = headers[i]
                date = dates[i]

                # extract relevant items
                header = header.text
                date = date.text.split('·')[1].strip()
                date = date.split(' ')[0]

                print(date)
                # print("date_tracker: ", date_tracker)

                if str(date) == start_date:
                    print("found 1: ", start_date)
                if str(date) == start_date and continue_scrape == False:
                    print("found 2: ", start_date)
                if str(date) == start_date and continue_scrape == False and str(date) <= date_tracker:
                    print("found 3: ", start_date)
                if str(date) == start_date and continue_scrape == False and str(date) <= date_tracker and header not in scraped_headers:
                    continue_scrape = True
                    scraped_headers.append(header)
                    scraped_dates.append(date)
                    date_tracker = str(date)
                    print("Scraping starts!...", header, date)

                if str(date) == end_date or str(date) == '01/02/2020' or str(date) == '12/30/2019' or str(date) == '12/29/2019':
                    print("Scraping ends!...", header, date)
                    driver.quit()
                    break

                if continue_scrape and str(date) <= date_tracker and header not in scraped_headers:
                    scraped_headers.append(header)
                    scraped_dates.append(date)
                    date_tracker = str(date)
                    print("Scraping...", header, date)
        except:
            driver.quit()
            break
    
    # Combine the two lists into a DataFrame
    df = pd.DataFrame({'date': scraped_dates, 'news_headline': scraped_headers})

    # Save the DataFrame as a CSV file
    df.to_csv(file_name, index=False)

In [56]:
# run the function
# start_phrase = "Rosenblatt Initiates Coverage On Zoom Video Comms with Neutral Rating, Announces Price Target of $75" # last article of dec 31 2022
# end_phrase = "How Zoom's Mission Could Help It Lower Its Biggest Expense" 


start_date = '12/30/2022' # last article of dec 31, 2022
end_date = '01/01/2020' # last article of jan 1, 2020
scraped_headers = []
scraped_dates = []
file_name = 'full_webull_zoom_stock_news.csv'
ticker = 'zm'

scrape_webull_headlines(ticker, start_date, end_date, scraped_headers, scraped_dates, file_name)

/var/folders/cf/28wm9krx5816hxprmj3lstxh0000gn/T/ipykernel_11467/1758405074.py:22: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  headers = driver.find_elements_by_class_name("csr14")
/var/folders/cf/28wm9krx5816hxprmj3lstxh0000gn/T/ipykernel_11467/1758405074.py:23: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  dates = driver.find_elements_by_class_name("csr15")


1h
12h
15h
17h
17h
17h
20h
1d
1d
1d
1d
1d
1d
4d
4d
4d
4d
4d
4d
4d
4d
5d
04/10
04/07
04/07
04/06
04/05
04/05
04/02
03/31
03/30
03/30
03/30
03/29
03/29
03/29
03/28
03/28
03/28
1h
12h
15h
17h
17h
17h
20h
1d
1d
1d
1d
1d
1d
4d
4d
4d
4d
4d
4d
4d
4d
5d
04/10
04/07
04/07
04/06
04/05
04/05
04/02
03/31
03/30
03/30
03/30
03/29
03/29
03/29
03/28
03/28
03/28
03/27
03/27
03/27
03/25
03/24
03/24
03/24
03/24
03/24
03/24
03/23
03/23
03/23
03/21
03/21
03/17
03/17
03/17
03/17
03/17
1h
12h
15h
17h
17h
17h
20h
1d
1d
1d
1d
1d
1d
4d
4d
4d
4d
4d
4d
4d
4d
5d
04/10
04/07
04/07
04/06
04/05
04/05
04/02
03/31
03/30
03/30
03/30
03/29
03/29
03/29
03/28
03/28
03/28
03/27
03/27
03/27
03/25
03/24
03/24
03/24
03/24
03/24
03/24
03/23
03/23
03/23
03/21
03/21
03/17
03/17
03/17
03/17
03/17
03/16
03/16
03/16
03/15
03/15
03/15
03/14
03/14
03/14
03/14
03/13
03/11
03/11
03/10
03/10
03/09
03/07
03/07
03/07
03/07
1h
12h
15h
17h
17h
17h
20h
1d
1d
1d
1d
1d
1d
4d
4d
4d
4d
4d
4d
4d
4d
5d
04/10
04/07
04/07
04/06
04/05
04/05
04/02
03/3